# Cómo ganar en la competencia 2

---

Esta es la parte 4 de la auxiliar, la parte 3 fue hacer un repaso de la arquitectura base propuesta para la competencia 2 y recomendaciones generales sobres los experimentos que pueden hacer para su informe.

Acá vamos a usar una librería que es super fácil de usar, llamda `flair`, para ver qué puntajes podemos obtener usando modelos preentrenados. Vamos a usar el set de validación de la competencia 2.

**IMPORTANTE 1**: Está demás decir que esto no lo pueden usar para la competencia 2, es decir, no pueden llegar y subir los resultados correctos del set de test y suponer que porque les fue super bien en la compentencia van a tener la mejor nota. Lo importante de la competencia es el informe que entregan con los experimentos que realizaron y sus reflexiones. Si los resultados que tienen en la competencia no se corresponden con los resultados que obtienen sus arquitecturas propuestas... no sé rick.

**IMPORTANTE 2**: Esto es menos importante eso sí. Este ejemplito es simplemente para demostrar las capacidades de modelos pre-entrenados y las utilidades que ofrece la libreria `flair`. También es posible entrenar los modelos que la librería tiene disponibles uno mismo, con sus propios datasets, pero según lo que te entendí, no es posible (al menos no salía en el tutorial) implementar modelos propios y usarlos en el framework, es más bien para reentrenar los mismos modelos probando datos distintos. Aunque hay que decir que también se puede jugar con los embeddings para el modelo, pero no implementar un nuevo modelo desde 0. De todas formas, si alguien sabe más de `flair`, cualquier comentario es bienvenido.

In [1]:
# Instalamos flair
%%capture --no-stderr
!pip install --upgrade flair torchtext

In [2]:
%%capture
!wget -nc https://github.com/dccuchile/CC6205/releases/download/Data/val_NER_esp.txt

In [3]:
# En esta parte simplemente voy a cargar el dataset con torchtext, porque
# me parece que no es tan simple cargar un solo split de un dataset con
# flair. Como que esta disennado para cargar un dataset completo (un corpus)
# con los tres splits al tiro.
from torchtext import data, datasets

TAG = data.Field(unk_token=None, pad_token=None)
TEXT = data.Field()

dataset = datasets.SequenceTaggingDataset(
    "val_NER_esp.txt",
    fields=[("text", TEXT), ("tags", TAG)],
    encoding="iso-8859-1",
    separator=" ",
)

In [4]:
# Aca cargamos un tagger, de tipo ner y entrenado con un corpus multilingual
# que incluia espannol
from flair.models import SequenceTagger
tagger = SequenceTagger.load("ner-multi")

2020-07-02 07:11:50,499 loading file /root/.flair/models/quadner-large.pt


In [5]:
# Printiemos el diccionario de tags del tagger, para ver si son compatibles
# con el esquema BIO del dataset.
# En este caso usan BIOES, asique vamos a tener que transformar los tags que
# produce el tagger mas adelante
print(tagger.tag_dictionary.get_items())

['<unk>', 'O', 'B-PER', 'E-PER', 'S-LOC', 'B-MISC', 'I-MISC', 'E-MISC', 'S-PER', 'B-ORG', 'E-ORG', 'S-ORG', 'I-ORG', 'B-LOC', 'E-LOC', 'S-MISC', 'I-PER', 'I-LOC', '<START>', '<STOP>']


In [6]:
# En esta celda tomamos una frase y la predecimos. Para crear un objeto
# `Sentence` en flair hay que pasarle una frase completa en forma de string
# y el tokenizador basico que ocupa simplemente hace str.split()
# Me parece muy raro que no permita formar frases a partir de tokens
# directamente, pero en la investigacion superficial que hice no aparecia como.
from pprint import pprint
from flair.data import Sentence
sentence = Sentence(" ".join(dataset[5].text))

# El tagger modifica la sentence `in-place`, es decir cambia los atributos
# del objeto en vez de retornar uno nuevo. Al crear la sentece, los tokens no
# tenian asociado un ner-tag y luego de realizar la prediccion si.
tagger.predict(sentence)
pprint(sentence.to_tagged_string())

('También participaron en el acto el consejero delegado de Telefónica <B-ORG> '
 'Internacional <E-ORG> , Antonio <B-PER> Viana <I-PER> Baptista <E-PER> ; el '
 'ministro de Comunicaciones <S-MISC> de Brasil <S-LOC> , Joao <B-PER> '
 'Pimienta <I-PER> da <I-PER> Veiga <E-PER> ; el presidente de la ANATEL '
 '<S-ORG> , Renato <B-PER> Navarro <I-PER> Guerreiro <E-PER> , y el secretario '
 'de Ciencia. <B-MISC> Tecnología <I-MISC> del <I-MISC> Estado <I-MISC> de '
 '<I-MISC> Sao <I-MISC> Paulo <E-MISC> , José <B-PER> Aníbal <I-PER> Peres '
 '<I-PER> de <I-PER> Pontes <E-PER> .')


In [7]:
# La sentence esta compuesta de tokens, y cada token tiene informacion de sus
# tags. Aca se muestran los primeros 5 tokens
pprint([token for token in sentence[:5]])

[Token: 1 También,
 Token: 2 participaron,
 Token: 3 en,
 Token: 4 el,
 Token: 5 acto]


In [8]:
# Despues de realizar la prediccion, cada token tiene asociado un tag y
# nivel de confianza en el tag asignado.

# El modelo esta muy seguro que la palabra "Tambien" no es una entidad
print(sentence[0], "\n", sentence[0].get_tag("ner"), "\n")
# El modelo esta bastante seguro que la palabra "Telefonica" es el comienzo
# de una organizacion
print(sentence[9], "\n", sentence[9].get_tag("ner"))

Token: 1 También 
 O (1.0) 

Token: 10 Telefónica 
 B-ORG (0.8297)


In [9]:
# Tambien podemos ver que a los tokens de la sentence no se les ha asigando
# un tag `pos` todavia. (Noten que no sale nada antes del 1.0)
print(sentence[0].get_tag("pos"))

 (1.0)


In [10]:
# Al tagger igual se le pueden pasar varias sentences para que las taguee,
# asi que le vamos a pasar todo el dataset para ver como le va
import torch
if not torch.cuda.is_available():
    raise Exception("Cambia el runtime si no quieres estar una tarde en esta celda")

sentences = [Sentence(" ".join(example.text)) for example in dataset]
tagger.predict(sentences, mini_batch_size=128, verbose=True)
pass # para que no printee el objeto que retorna el statement anterior

Inferencing on batch 14: 100%|██████████| 15/15 [00:26<00:00,  1.75s/it]


In [11]:
# Ahora realizamos la evaluacion, usando el f1_score de entidad predicha
# a nivel de token
from sklearn.metrics import f1_score

# Un mapping para convertir de BIOES a BIO
mapping = (
    ("E", "I"),
    ("S", "B")
)

# Una funcioncita para convertir de BIOES a BIO
def to_bio(tag):    
    for t_in, t_out in mapping:
        tag = tag.replace(t_in, t_out, 1)
    return tag


TAG.build_vocab(dataset)
y_true = [tag for example in dataset for tag in example.tags]
y_pred = [to_bio(token.get_tag("ner").value) for sentence in sentences for token in sentence]

print("El f1_score del set de validacion, asi sin hacer nada, es:")
print(f1_score(y_true, y_pred, labels=[tag for tag in TAG.vocab.itos if tag != "O"], average="micro"))


El f1_score del set de validacion, asi sin hacer nada, es:
0.8137920274592402
